In [2]:
import json
from lxml import objectify
from selenium import webdriver
from bs4 import BeautifulSoup

import urllib
from urllib.request import urlopen
from urllib.error import HTTPError
from datetime import datetime
from datetime import timedelta
import time
import pandas
import numpy
import re

## Links and params

In [3]:
## LTA
headers = { 'AccountKey' : 'LBdfS4+RSEi4witEa6RjjQ==',
            'UniqueUserID' : '812eb03f-f3a1-421c-96de-069c59844bbb',
            'accept' : 'application/json'}
uri = 'http://datamall2.mytransport.sg/'
path_TSB = 'ltaodataservice/TrafficSpeedBands'
path_TI = 'ltaodataservice/TrafficIncidents'
path_FTL = 'ltaodataservice/FaultyTrafficLights'
path_ETT = 'ltaodataservice/EstTravelTimes'
path_RO = 'ltaodataservice/RoadOpenings'
path_RW = 'ltaodataservice/RoadWorks'
## NEA links and auth params
key = '781CF461BB6606ADC767F3B357E848ED4A6709A0B39C63D6'
urlNowcast = 'http://api.nea.gov.sg/api/WebAPI/?dataset=2hr_nowcast&keyref=' + key
urlWarning = 'http://api.nea.gov.sg/api/WebAPI/?dataset=heavy_rain_warning&keyref=' + key
## Meteorological Service Singapore Rainfall observations
urlRainfall = 'http://www.weather.gov.sg/weather-currentobservations-rainfall'

### Test LTA data EPN

In [4]:
# Traffic Speed Band data - updates every 5 minutes
request = urllib.request.Request(uri + path_TSB, headers=headers)
try:
    response = urlopen(request).read().decode('utf-8')
except HTTPError as e:
    print (e)
else:
    jsonObj = json.loads(response)
    messages = jsonObj.get("value")
    dfTSB = pandas.DataFrame(messages)

In [30]:
dfTSB = dfTSB.set_index('LinkID')
dfTSB.index.name = None
dfTSB.head()

,Location,MaximumSpeed,MinimumSpeed,RoadCategory,RoadName,SpeedBand
103098943,1.3805895432850994 103.74201730473243 1.380040...,59,40,D,CHOA CHU KANG AVENUE 3,3
103076063,1.399027473240379 103.7553182996927 1.39866097...,59,40,B,WOODLANDS ROAD,3
103030883,1.3742439017982357 103.88529975827204 1.373869...,39,20,D,HOUGANG AVENUE 4,2
104003992,1.3163287378822295 103.76133268167833 1.315260...,59,40,F,CLEMENTI AVENUE 6,3
103110036,1.333534171783063 103.88445104036495 1.3344802...,39,20,E,HARVEY ROAD,2


In [6]:
#dfTSB2 = dfTSB#.set_index('LinkID')
#dfTSB2.index.name = None
dfTSB2.head()

,Location,MaximumSpeed,MinimumSpeed,RoadCategory,RoadName,SpeedBand
103098943,1.3805895432850994 103.74201730473243 1.380040...,39,20,D,CHOA CHU KANG AVENUE 3,2
103076063,1.399027473240379 103.7553182996927 1.39866097...,59,40,B,WOODLANDS ROAD,3
103030883,1.3742439017982357 103.88529975827204 1.373869...,39,20,D,HOUGANG AVENUE 4,2
104003992,1.3163287378822295 103.76133268167833 1.315260...,59,40,F,CLEMENTI AVENUE 6,3
103110036,1.333534171783063 103.88445104036495 1.3344802...,,60,E,HARVEY ROAD,4


In [31]:
neRows = (dfTSB != dfTSB2).any(1)
neRows
ne_stacked = (dfTSB != dfTSB2).stack()
ne_stacked
changed = ne_stacked[ne_stacked]
changed.index.names = ['id', 'col']
changed
diff_loc = numpy.where(dfTSB != dfTSB2)
changed_from = dfTSB.values[diff_loc]
changed_to = dfTSB2.values[diff_loc]
diff = pandas.DataFrame({'from': changed_from, 'to': changed_to}, index=changed.index)
diff.head()

from  to
id        col                  
103098943 MaximumSpeed   59  39
          MinimumSpeed   40  20
          SpeedBand       3   2
103110036 MaximumSpeed   39    
          MinimumSpeed   20  60

In [79]:
for id, row in diff.iterrows():
    if (id[1] == 'SpeedBand'):
        print(row['from'], row['to']) 
        print(id[0])
        #print(dfTSB2.ix(id[0]))
        print(dfTSB2.get_value(id[0],'Location'))

3 2
103098943
1.3805895432850994 103.74201730473243 1.3800400299576012 103.74120855605975
2 4
103110036
1.333534171783063 103.88445104036495 1.334480240662004 103.88423757172991
2 3
103089006
1.3468683821422813 103.69025703809406 1.3466193981361332 103.6903389382926
2 1
103037283
1.354538472093295 103.86279521974849 1.3554224742291467 103.863192917283
2 4
103070015
1.3777681206363968 103.94139850987841 1.378867186631845 103.94152708219333
2 3
103031346
1.4026080634287352 103.81525433456433 1.402689645719599 103.81640482532244
2 4
103026994
1.3806647169807298 103.86241340163942 1.3815437998182825 103.86275797975088
2 3
103067603
1.3155700538658663 103.76502558167256 1.3146394741932022 103.76542384414495
3 4
103028396
1.354555356422381 103.98908738997977 1.3535347292627729 103.98854931143833
3 2
103011039
1.2979413745180766 103.8858495015343 1.2993701686587462 103.88595357348129
3 2
103080426
1.3789273339780308 103.8766121640118 1.378711751253142 103.8766364665296
4 3
103095079
1.3429777

In [33]:
if(dfTSB.equals(dfTSB2)):
    print('same')
if not (dfTSB.equals(dfTSB2)):
    print('diff')

diff


In [10]:
# ESTIMATED TRAVEL TIMES data - updates every 5 minutes
request = urllib.request.Request(uri + path_ETT, headers=headers)
try:
    response = urlopen(request).read().decode('utf-8')
except HTTPError as e:
    print (e)
else:
    jsonObj = json.loads(response)
    messages = jsonObj.get("value")
    dfETT = pandas.DataFrame(messages)

In [11]:
dfETT.head(2)

,Direction,EndPoint,EstTime,FarEndPoint,Name,StartPoint
0,1,TELOK BLANGAH RD,2,TUAS CHECKPOINT,AYE,AYE/MCE INTERCHANGE
1,1,LOWER DELTA RD,1,TUAS CHECKPOINT,AYE,TELOK BLANGAH RD


In [59]:
# collect Traffic Incidents data - updates every 2 minutes
request = urllib.request.Request(uri + path_TI, headers=headers)
try:
    response = urlopen(request).read().decode('utf-8')
except HTTPError as e:
    print (e)
else:
    jsonObj = json.loads(response)
    messages = jsonObj.get("value")
    dfTI = pandas.DataFrame(messages)

In [60]:
dfTI.head()

,Latitude,Longitude,Message,Type
0,1.296161,103.851726,(11/11)23:05 Roadworks on Bras Basah Road (tow...,Roadwork
1,1.321684,103.891215,(11/11)23:03 Roadworks on Paya Lebar Road (tow...,Roadwork
2,1.295772,103.851747,(11/11)22:58 Roadworks on Victoria Street (tow...,Roadwork
3,1.361200,103.858495,(11/11)22:58 Roadworks on CTE (towards AYE) be...,Roadwork
4,1.276845,103.848669,(11/11)22:49 Roadworks on Shenton Way (towards...,Roadwork


In [49]:
#dfTI2 = dfTI
dfTI2.head()

,Latitude,Longitude,Message,Type
0,1.294434,103.866525,(11/11)21:39 Roadworks on ECP (towards Changi ...,Roadwork
1,1.277375,103.848986,(11/11)21:38 Roadworks on Shenton Way (towards...,Roadwork
2,1.296506,103.850972,(11/11)21:35 Roadworks on Stamford Road (towar...,Roadwork
3,1.321425,103.856028,(11/11)21:28 Heavy Traffic on CTE (towards SLE...,Heavy Traffic
4,1.313303,103.874861,(11/11)21:27 Roadworks on Sims Way near Sims A...,Roadwork


In [107]:
diffTI = pandas.concat([dfTI, dfTI2, dfTI2]).drop_duplicates(keep=False)
#print(diffTI)
#print(datetime.now().strftime("%H:%M") + " TRAFFIC INCIDENCE")
patternTime = re.compile('[0-9]+:[0-9]+')
patternMsg = re.compile('^\([0-9]+/[0-9]+\)[0-9]+:[0-9]+([A-Za-z ()]+)')
for id, row in diffTI.iterrows():
    #print(row['Message'])
    print(patternTime.findall(row['Message'])[0] + patternMsg.findall(row['Message'])[0])

23:05 Roadworks on Bras Basah Road (towards Raffles Boulevard) after Prinsep Street
23:03 Roadworks on Paya Lebar Road (towards Jalan Afifi) after PIE
22:58 Roadworks on Victoria Street (towards Bras Basah Road) after Stamford Road
22:58 Roadworks on CTE (towards AYE) before AMK Ave 
22:49 Roadworks on Shenton Way (towards Shenton Way) after Central Boulevard
22:48 Roadworks on ECP (towards City) after Rochor Rd Exit
22:40 Heavy Traffic on CTE (towards SLE) between PIE(Changi) Exit and AMK Ave 
22:37 Roadworks on Grange Road (towards Orchard Boulevard) after Cuscaden Road
22:35 Roadworks on Changi South Avenue 
22:33 Roadworks on PIE (towards Changi Airport) after Stevens Rd Exit
22:31 Roadworks on Cairnhill Circle near Cairnhill Circle
22:29 Roadworks on Collyer Quay (towards Shenton Way) after Fullerton Road
22:29 Roadworks on PIE (towards Changi Airport) at Tampines Ave 
22:29 Roadworks on Collyer Quay (towards Guillemard) after Marina Boulevard


In [111]:
# collect Nowcast data - updates every 30 mins
request = urllib.request.Request(urlNowcast)
try:
    parsed = objectify.parse(urlopen(request))
except HTTPError as e:
    print (e)
else:
    parsed = objectify.parse(urlopen(request))
    root = parsed.getroot()
    dfNowcast = pandas.DataFrame(columns=['Forecast', 'Latitude', 'Longitude', 'Name'])
    for area in root.iter('area'):
        dfNowcast = dfNowcast.append({'Forecast':area.get('forecast'),'Latitude':area.get('lat'),'Longitude':area.get('lon'),'Name':area.get('name')}, ignore_index=True)

In [112]:
dfNowcast.head(2)

,Forecast,Latitude,Longitude,Name
0,FA,1.37500000,103.83900000,Ang Mo Kio
1,FA,1.32100000,103.92400000,Bedok


In [113]:
dfNowc = dfNowcast.set_index('Name')
dfNowc.index.name = None
print(dfNowc.head())

            Forecast    Latitude     Longitude
Ang Mo Kio        FA  1.37500000  103.83900000
Bedok             FA  1.32100000  103.92400000
Bishan            FA  1.35077200  103.83900000
Boon Lay          FA  1.30400000  103.70100000
Bukit Batok       FA  1.35300000  103.75400000


In [110]:
#dfNowc2 = dfNowc
dfNowc.head(3)

,Forecast,Latitude,Longitude
Ang Mo Kio,CL,1.37500000,103.83900000
Bedok,CL,1.32100000,103.92400000
Bishan,CL,1.35077200,103.83900000


In [109]:
for id, row in dfNowc.iterrows():
    if (row['Forecast'] == "HR"):
        print ("Heavy Rain Forecast in " + id)

In [114]:
neRows = (dfNowc != dfNowc2).any(1)
ne_stacked = (dfNowc != dfNowc2).stack()
changed = ne_stacked[ne_stacked]
changed.index.names = ['id', 'col']
diff_loc = numpy.where(dfNowc != dfNowc2)
changed_from = dfNowc.values[diff_loc]
changed_to = dfNowc2.values[diff_loc]
diff = pandas.DataFrame({'from': changed_from, 'to': changed_to}, index=changed.index)
diff.head()

,,from,to
id,col,,
Ang Mo Kio,Forecast,FA,CL
Bedok,Forecast,FA,CL
Bishan,Forecast,FA,CL
Boon Lay,Forecast,FA,CL
Bukit Batok,Forecast,FA,CL


In [119]:
for id, row in diff.iterrows():
    if (row['from'] == "FA"):
        print ("Heavy Rain Forecast in " + id[0])

Heavy Rain Forecast in Ang Mo Kio
Heavy Rain Forecast in Bedok
Heavy Rain Forecast in Bishan
Heavy Rain Forecast in Boon Lay
Heavy Rain Forecast in Bukit Batok
Heavy Rain Forecast in Bukit Merah
Heavy Rain Forecast in Bukit Panjang
Heavy Rain Forecast in Bukit Timah
Heavy Rain Forecast in Central Water Catchment
Heavy Rain Forecast in Changi
Heavy Rain Forecast in Choa Chu Kang
Heavy Rain Forecast in Clementi
Heavy Rain Forecast in City
Heavy Rain Forecast in Geylang
Heavy Rain Forecast in Hougang
Heavy Rain Forecast in Jalan Bahar
Heavy Rain Forecast in Jurong East
Heavy Rain Forecast in Jurong Island
Heavy Rain Forecast in Jurong West
Heavy Rain Forecast in Kallang
Heavy Rain Forecast in Lim Chu Kang
Heavy Rain Forecast in Mandai
Heavy Rain Forecast in Marine Parade
Heavy Rain Forecast in Novena
Heavy Rain Forecast in Pasir Ris
Heavy Rain Forecast in Paya Lebar
Heavy Rain Forecast in Pioneer
Heavy Rain Forecast in Pulau Tekong
Heavy Rain Forecast in Pulau Ubin
Heavy Rain Forecast in 

In [17]:
# collect Heavy Rain Warning data - updates every hour
request = urllib.request.Request(urlWarning)
try:
    parsed = objectify.parse(urlopen(request))
except HTTPError as e:
    print (e)
else:
    parsed = objectify.parse(urlopen(request))
    root = parsed.getroot()
    HRWmsg = root.item.warning.text
    HRWmsg = HRWmsg.strip()
    HRWtime = root.item.issue_datentime.text
    HRWtime = HRWtime.strip()

In [18]:
if('NIL' in HRWmsg):
    print('No warnings')
else:
    print(HRWmsg)

No warnings


In [100]:
# collect Rainfall data - updates every 10 mins
driver = webdriver.PhantomJS(executable_path='../bin/phantomjs')
try:
    driver.get(urlRainfall)
except HTTPError as e:
    print (e)
else:
    # wait for page load
    time.sleep(2)
    html30mins = driver.page_source
    bsObj = BeautifulSoup(html30mins,"lxml")
    # Get data timestamp
    element = bsObj.find("img", {"id":"basemap"})
    pattern = re.compile('[0-9]+:[0-9]+')
    rainfallTimestamp = pattern.findall(element.attrs['src'])[0]
    # load 30 mins rainfall data
    dataset30mins =  bsObj.findAll("",{"class":"sgr"})
    dfRainfall = pandas.DataFrame(columns=['StationId', 'rain30mins', 'Station'])
    for data in dataset30mins:
        station = patternStation.findall(data.get('data-content'))
        dfRainfall = dfRainfall.append({'StationId':data.get('id'), 'rain30mins':data.get_text(), 'Station':station[0]},ignore_index=True)

In [101]:
dfRainfall.head(2)

,StationId,rain30mins,Station
0,S07,0,Macritchie Reservoir
1,S08,0,Lower Peirce Reservoir


In [102]:
dfRain = dfRainfall.set_index('StationId')
dfRain.index.name = None
dfRain.ix['S113']
dfRain.head()

,rain30mins,Station
S07,0,Macritchie Reservoir
S08,0,Lower Peirce Reservoir
S101,0,Jurong (North)
S104,0,Admiralty
S106,0,Pulau Ubin


In [104]:
for id, row in dfRain.iterrows():
    if (float(row['rain30mins']) > 7.5):
        print ("Heavy Rain in " + row['Station'] + ", drive carefully.")

In [99]:
#for data in dataset30mins:
#        dfRainfall = dfRainfall.append({'StationId':data.get('id'), 'rain30mins':data.get_text()},ignore_index=True)
patternStation = re.compile('<strong>([A-Za-z ()]+)</strong>')
#Stations = patternStation.findall(element.attrs['src'])[0]
for data in dataset30mins:
    Station = patternStation.findall(data.get('data-content'))
    print(Station)
#print(Stations[0])

['Macritchie Reservoir']
['Lower Peirce Reservoir']
['Jurong (North)']
['Admiralty']
['Pulau Ubin']
['East Coast Parkway']
['Marina Barrage']
['Ang Mo Kio']
['Choa Chu Kang (West)']
['Serangoon North']
['Lim Chu Kang']
['Marine Parade']
['Choa Chu Kang (Central)']
['Tuas South']
['Pasir Panjang']
['Jurong Island']
['Dhoby Ghaut']
['Nicoll Highway']
['Botanic Garden']
['Choa Chu Kang (South)']
['Khatib']
['Whampoa']
['Changi']
['Jurong Pier']
['Ulu Pandan']
['Mandai']
['Tai Seng']
['Jurong (West)']
['Upper Thomson']
['Clementi']
['Buangkok']
['Sentosa Island']
['Chai Chee']
['Boon Lay (West)']
['Bukit Panjang']
['Kranji Reservoir']
['Upper Peirce Reservoir']
['Kent Ridge']
['Queenstown']
['Tanjong Katong']
['Somerset (Road)']
['Punggol']
['Simei']
['Toa Payoh']
['Tuas']
['Bukit Timah']
['Yishun']
['Buona Vista']
['Pasir Ris (Central)']
